# Cleaning Data from Customer List 

In [1]:
import numpy as np
import pandas as pd

In [2]:
with open("customer_list_updated.csv", "r") as f:
    print(f.readlines(150))

['ï»¿cust_id|date|time|name|email|phone|sms-opt-out \n', '1|2023-03-15|08:45:12|Rachel|rachel@centralperk.coffee|212-555-1001|N\n', '2|2023-05-22|12:30:45|R&! Geller|rossg@centralperk.coffee|212-555-1002|N\n']


Used the open() function to read the file and see what I'm working with. Immediately noticed values were separated with pipes "**|**" instead of commas "**,**" .
This is something I will have to specify as the delimiter when I import it as dataframe.

Also, noticed a BOM(Byte-Order-Mark) **ï»¿** at the beginning of the code, which isn't shown in a notepad or VSC when I opened it and it's used for UTF files. This will cause an issue when importing if I dont encode it as UTF-8. 

For the sake of this project, since I need to cleanup the data, I will remove the BOM and replace the "|" with "," so it's simpler to import into a dataframe without needing to specify the delimiter and encoding it as a UTF-8

In [3]:
with open("customer_list_updated.csv", "r") as f:
    content = f.read()

# Manually remove BOM (first 3 indexes)
content = content[3:] 

# Replace "|" with ","
content = content.replace("|", ",")

# Write the new content back to a new file
with open("customer_list_ready.csv", "w") as f:
    f.write(content)

# Print to verify that BOM is removed and values are separated by commas
print(content)

cust_id,date,time,name,email,phone,sms-opt-out 
1,2023-03-15,08:45:12,Rachel,rachel@centralperk.coffee,212-555-1001,N
2,2023-05-22,12:30:45,R&! Geller,rossg@centralperk.coffee,212-555-1002,N
3,2023-07-09,18:15:27,Monica Geller,chefmonica@centralperk.coffee,212-555-1003,N
4,2023-09-01,21:05:33,Chandler Bing,chandlerb@centralperk.coffee,212-555-1004,Y
5,2023-11-18,14:22:10,Joey,howyoudoing@centralperk.coffee,212-555-1005,N
6,2024-01-05,10:55:49,P&! Buffay,smellycat@centralperk.coffee,212-555-1006,N
7,2024-02-14,16:40:05,Gunther,gunther@centralperk.coffee,212-555-1007,N
8,2023-04-20,09:15:30,Janice,ohmygod@centralperk.coffee,212-555-1008,N
9,2023-06-30,13:50:55,Mike H.^,mike@centralperk.coffee,212-555-1009,N
10,2023-08-25,17:25:10,Emily,emily@centralperk.coffee,212-555-1010,N
11,2023-04-12,09:20:30,Walter White,heisenberg@lospolloshermanos.biz,505-555-2001,N
12,2023-06-25,13:45:50,Jesse Pinkman,jesse@lospolloshermanos.biz,505-555-2002,N
13,2023-08-03,17:30:15,Skyler W.^,skyler@lospolloshe

In [4]:
# Creating a dataframe with new csv file, specifying the 'cust_id' column as index
df = pd.read_csv("customer_list_ready.csv", index_col='cust_id')

In [5]:
# Displaying the first 10 rows of df
df.head(10)

,date,time,name,email,phone,sms-opt-out
cust_id,,,,,,
1,2023-03-15,08:45:12,Rachel,rachel@centralperk.coffee,212-555-1001,N
2,2023-05-22,12:30:45,R&! Geller,rossg@centralperk.coffee,212-555-1002,N
3,2023-07-09,18:15:27,Monica Geller,chefmonica@centralperk.coffee,212-555-1003,N
4,2023-09-01,21:05:33,Chandler Bing,chandlerb@centralperk.coffee,212-555-1004,Y
5,2023-11-18,14:22:10,Joey,howyoudoing@centralperk.coffee,212-555-1005,N
6,2024-01-05,10:55:49,P&! Buffay,smellycat@centralperk.coffee,212-555-1006,N
7,2024-02-14,16:40:05,Gunther,gunther@centralperk.coffee,212-555-1007,N
8,2023-04-20,09:15:30,Janice,ohmygod@centralperk.coffee,212-555-1008,N
9,2023-06-30,13:50:55,Mike H.^,mike@centralperk.coffee,212-555-1009,N


***
Noticed right away in the first 10 rows that the names have special characters:
**&!** for first name initials and **.^** for last name initials

I will create a code to find all names and index# to see which characters I will need to remove
***

In [6]:
# Loop through each row in the 'name' column
for index, name in df['name'].items():
    valid_name = True

    for char in name:
        # Check if the character is not a letter and not a space
        if not (char.isalpha() or char == ' '):
            valid_name = False
            break

    # If the name is invalid, print the row's index and name
    if not valid_name:
        print(f"Index {index}: {name}")

Index 2: R&! Geller
Index 6: P&! Buffay
Index 9: Mike H.^
Index 13: Skyler W.^
Index 15: H&! Schrader
Index 17: Mike E.^
Index 18: Marie S.^
Index 20: Lydia R.^
Index 22: Daenerys T.^
Index 23: Tyrion L.^
Index 25: Sansa S.^
Index 26: Cersei L.^
Index 27: J&! Lannister
Index 29: Samwell T.^
Index 31: Homer S.^
Index 34: Lisa S.^
Index 37: Mr&! Burns
Index 42: Jim H.^
Index 45: Angela M.^
Index 46: Kevin M.^
Index 47: Stanley H.^
Index 48: Andy B.^
Index 49: Oscar M.^
Index 52: Mike W.^
Index 53: Dustin H.^
Index 54: Lucas S.^
Index 55: Will B.^
Index 58: Tom H.^
Index 59: April L.^
Index 61: Jake P.^
Index 62: Amy S.^
Index 63: Terry J.^
Index 64: Rosa D.^
Index 65: Charles B.^
Index 67: Britta P.^
Index 71: Shirley B.^
Index 72: Pierce H.^
Index 78: Saul B.^
Index 81: Jessica B.^
Index 82: Dana B.^
Index 84: Fara S.^
Index 91: Beverley L.^
Index 94: Vince D'Angelo
Index 96: Daria M.^
Index 98: Quinn M.^
Index 100: Kevin T.^
Index 101: Brittany T.^
Index 102: Jodie L.^
Index 103: Mack 

***
By creating and running this function, I noticed there were many names that had characters. 

Most of them had the **&!** characters in the first name for initials and **.^** for last name initials. 

I also noticed for row 301, Miles **O[']Brien** had brackets around the apostrophe for his last name so I will specify to remove the brackets.

There were also some customers that had a correct special character: hyphens **-** , so I will make sure not to remove those
***

In [7]:
# Remove specific unwanted characters in the 'name' column like "&!", "^", and "[']"
for char in ['&!']:
    df['name'] = df['name'].str.replace(char, '.')   # Replacing with a period

for char in ['^']:
    df['name'] = df['name'].str.replace(char, '') 

for char in ["[']"]:
    df['name'] = df['name'].str.replace(char, "'")   # Removing the square brackets and reinstating an apostrophe

In [8]:
# Checking the first 10 rows to see if it changed the characters
df.head(10) 

,date,time,name,email,phone,sms-opt-out
cust_id,,,,,,
1,2023-03-15,08:45:12,Rachel,rachel@centralperk.coffee,212-555-1001,N
2,2023-05-22,12:30:45,R. Geller,rossg@centralperk.coffee,212-555-1002,N
3,2023-07-09,18:15:27,Monica Geller,chefmonica@centralperk.coffee,212-555-1003,N
4,2023-09-01,21:05:33,Chandler Bing,chandlerb@centralperk.coffee,212-555-1004,Y
5,2023-11-18,14:22:10,Joey,howyoudoing@centralperk.coffee,212-555-1005,N
6,2024-01-05,10:55:49,P. Buffay,smellycat@centralperk.coffee,212-555-1006,N
7,2024-02-14,16:40:05,Gunther,gunther@centralperk.coffee,212-555-1007,N
8,2023-04-20,09:15:30,Janice,ohmygod@centralperk.coffee,212-555-1008,N
9,2023-06-30,13:50:55,Mike H.,mike@centralperk.coffee,212-555-1009,N


In [9]:
# Using .info() to see how many rows are there and any other information I need
df.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 521 entries, 1 to 521
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          521 non-null    object
 1   time          521 non-null    object
 2   name          521 non-null    object
 3   email         521 non-null    object
 4   phone         520 non-null    object
 5   sms-opt-out   520 non-null    object
dtypes: object(6)
memory usage: 28.5+ KB


***
From the info summary, I can deduce that there are 521 entries (rows).
The "phone" and "sms-opt-out" columns are only showing 520 non-null entries so they probably have one NaN value each
***

In [10]:
# Summing up all null values for each column
df.isnull().sum()

date            0
time            0
name            0
email           0
phone           1
sms-opt-out     1
dtype: int64

In [11]:
# Locating which row has the NaN in the phone column
df.loc[df['phone'].isna()] 

,date,time,name,email,phone,sms-opt-out
cust_id,,,,,,
301,2023-09-12,12:15:40,Miles O'Brien,obrien@ds9.starfleet,NaN,NaN


***
Found out that row 301 has both NaN values I was looking for. I will go ahead and add values into the phone and sms columns for row 301. I rather do this than drop the row since it still has valuable data.

***

In [12]:
df.loc[301, 'phone'] = '000-000-0000'  

In [13]:
df.loc[301, 'sms-opt-out'] = 'Y'  # Putting Y for opt-out since it's an invalid #

In [14]:
df.loc[[301]]

,date,time,name,email,phone,sms-opt-out,sms-opt-out
cust_id,,,,,,,
301,2023-09-12,12:15:40,Miles O'Brien,obrien@ds9.starfleet,000-000-0000,NaN,Y


***
Upon reviewing row 301 after my changes, I noticed that I created another column when I wanted to input Y for sms-opt-out. 

Although they're the same spelling, I think it created an additional column since the og must have a space before or after
***

In [15]:
# Checking the og column for the name
df.columns[5]

'sms-opt-out '

***
Confirmed that the original column for sms-opt-out had a space after so I will delete the column I created and delete the space in the original column
***

In [16]:
# Dropping the column I made
df.drop('sms-opt-out', axis=1, inplace=True)

In [17]:
# Stripping the space in all the columns, just in case
df.columns = df.columns.str.strip()

In [18]:
# Confirming column has no space
df.columns[5]

'sms-opt-out'

In [19]:
# Running my code again 
df.loc[301, 'sms-opt-out'] = 'Y'  # Putting Y for opt-out since it's an invalid #

In [20]:
df.loc[[301]]

,date,time,name,email,phone,sms-opt-out
cust_id,,,,,,
301,2023-09-12,12:15:40,Miles O'Brien,obrien@ds9.starfleet,000-000-0000,Y


***
It worked! I will now go column by column performing a few codes to see if there are any discrepancies I need to correct
***

In [21]:
# Starting with the date column

# Start by stripping any spaces
df['date'] = df['date'].str.strip()

# Looping through each row in the 'date column and check if the date format is correct
for index, date in df['date'].items():
    valid_date = True

    # If date doesn't match format below, it will raise the flag false
    if len(date) != 10 or date[4] != '-' or date[7] != '-':
        valid_date = False

    # Checking if dates are digits only (no special characters besides hyphen and no letters)
    if not date[:4].isdigit() or not date[5:7].isdigit() or not date[8:].isdigit():
        valid_time = False
        
    # If any came out invalid, it will print below
    if not valid_date:
        print(f"Index {index}: {date}")

In [22]:
# Since nothing printed, all dates were inputted correctly

df.head()

,date,time,name,email,phone,sms-opt-out
cust_id,,,,,,
1,2023-03-15,08:45:12,Rachel,rachel@centralperk.coffee,212-555-1001,N
2,2023-05-22,12:30:45,R. Geller,rossg@centralperk.coffee,212-555-1002,N
3,2023-07-09,18:15:27,Monica Geller,chefmonica@centralperk.coffee,212-555-1003,N
4,2023-09-01,21:05:33,Chandler Bing,chandlerb@centralperk.coffee,212-555-1004,Y
5,2023-11-18,14:22:10,Joey,howyoudoing@centralperk.coffee,212-555-1005,N


In [23]:
# Running a similar code for the time column

# Start by stripping any spaces
df['time'] = df['time'].str.strip()

# Looping through each row in the 'time' column and check if the time format is correct
for index, time in df['time'].items():
    valid_time = True

    # Check if the time format is exactly 'HH:MM:SS' (length of 8, and correct position of ':')
    if len(time) != 8 or time[2] != ':' or time[5] != ':':
        valid_time = False

    # Check if each part of the time is numeric
    if not time[:2].isdigit() or not time[3:5].isdigit() or not time[6:].isdigit():
        valid_time = False

    # If any row is invalid, it will print below
    if not valid_time:
        print(f"Index {index}: {time}")

In [24]:
# Again, since nothing printed, all dates were inputted correctly:

df.head()

,date,time,name,email,phone,sms-opt-out
cust_id,,,,,,
1,2023-03-15,08:45:12,Rachel,rachel@centralperk.coffee,212-555-1001,N
2,2023-05-22,12:30:45,R. Geller,rossg@centralperk.coffee,212-555-1002,N
3,2023-07-09,18:15:27,Monica Geller,chefmonica@centralperk.coffee,212-555-1003,N
4,2023-09-01,21:05:33,Chandler Bing,chandlerb@centralperk.coffee,212-555-1004,Y
5,2023-11-18,14:22:10,Joey,howyoudoing@centralperk.coffee,212-555-1005,N


In [25]:
# Will skip the name column since it's already been cleaned and move on to the email column:

# Loop through each email in the 'email' column
for index, email in df['email'].items():
    # Remove extra spaces (leading and trailing)
    cleaned_email = email.strip()

    # Converting email to lowercase
    cleaned_email = cleaned_email.lower()

    # Check if email doesn't contain '@' symbol AND a '.'
    if not'@' in cleaned_email or not '.' in cleaned_email: 
         print(f"Index {index}: {email}")

In [26]:
# Nothing printed so email column is clean so I will move onto the phone column:

# Start by stripping any spaces
df['phone'] = df['phone'].str.strip()

# Loop through each phone number in the 'phone' column
for index, phone in df['phone'].items():
    valid_phone = True
    
    # Check if the phone number has exactly 12 characters (fitting this format: 123-456-7890)
    if len(phone) != 12:
        valid_phone = False

    # Step 2: Check if dashes are in the correct places
    if phone[3] != '-' or phone[7] != '-':
        valid_phone = False
    
    # Step 3: Check if the characters are numbers where they should be
    if not phone[:3].isdigit() or not phone[4:7].isdigit() or not phone[8:].isdigit():
        valid_phone = False

    # If the phone is invalid, print it
    if not valid_phone:
        print(f"Index {index}: {phone}")

Index 286: 1701-555-2201
Index 287: 1701-555-2202
Index 288: 1701-555-2203
Index 289: 1701-555-2204
Index 290: 1701-555-2205
Index 291: 1701-555-2206
Index 292: 1701-555-2207
Index 293: 1701-555-2208
Index 294: 1701-555-2209
Index 295: 1701-555-2210


***
Finally, found an error in the phone column from rows 286-295! 

All phone #'s have a 1 before the number (specifying the country code).
I will remove it to keep things uniform and clean
***

In [27]:
# Loop through each phone number in the 'phone' column
for index, phone in df['phone'].items():
    # Check to find the phone numbers that are 13 characters long
    if len(phone) == 13:
        # Removing the first character and update the phone number in the DataFrame
        df.loc[index, 'phone'] = phone[1:]

# printing rows from 286 to 295 to confirm
df.loc[286:295]

,date,time,name,email,phone,sms-opt-out
cust_id,,,,,,
286,2023-01-01,08:00:00,Jean-Luc Picard,picard@enterprise.starfleet,701-555-2201,N
287,2023-02-14,12:25:15,William Riker,riker@enterprise.starfleet,701-555-2202,N
288,2023-03-27,15:50:30,Data,data@enterprise.starfleet,701-555-2203,N
289,2023-05-10,19:15:45,Geordi La Forge,geordi@enterprise.starfleet,701-555-2204,N
290,2023-06-23,07:40:00,Worf,worf@enterprise.starfleet,701-555-2205,Y
291,2023-08-06,11:05:15,Deanna Troi,troi@enterprise.starfleet,701-555-2206,N
292,2023-09-19,14:30:30,Beverly Crusher,crusher@enterprise.starfleet,701-555-2207,N
293,2023-11-02,17:55:45,Wesley Crusher,wesley@enterprise.starfleet,701-555-2208,N
294,2023-12-15,21:20:00,Tasha Yar,tasha@enterprise.starfleet,701-555-2209,N


***
Great! Phones have been updated and cleaned. Last, but not least, I will clean the last column
***

In [28]:
# Start by stripping any spaces
df['sms-opt-out'] = df['sms-opt-out'].str.strip()

# Replace NaN values with 'Y' as default for customers to opt-out
df['sms-opt-out'] = df['sms-opt-out'].fillna('Y')

# Converting all entries to uppercase for uniformity
df['sms-opt-out'] = df['sms-opt-out'].str.upper()

# Checking for invalid entries
for index, value in df['sms-opt-out'].items():
    if value not in ['Y', 'N']:
        print(f"Index {index}: {value}")

All columns have cleaned, I will now check for any duplicates, just in case

In [29]:
# Check for duplicates in the entire DataFrame
print(df[df.duplicated()])

Empty DataFrame
Columns: [date, time, name, email, phone, sms-opt-out]
Index: []


***
Great! All columns have cleaned!

As a summary, I initally removed the BOM, and changed the delimiter from pipes **|** to commas **,**

Then, I removed all the special characters in the names since that was what caught my eye first.

I proceeded to check the info() summary to see how many rows I was working with and see if there were any nulls 

Removed the nulls in the phone and sms-opt-out column and replaced them with values, while also stripping spaces in the columns to not create new columns by accident

I then, checked column by column if there were any formatting issues and found one last error in the phones column a set of numbers with 13 characters instead of 12.

Finally, stripped it 12 and the data has been cleaned
***